In [ ]:
#@title Placeholder for description
"""
from pprint import pprint
p1.df_stkLim[['strategy', 'strategydescription', 'tradePlanTemplate', 'lifecycle', 'setup', 'setupFilters', 'options', 'entries', 'exits']].iloc[700:900]
selected = 'lifecycle'
for selected in ['strategy', 'strategydescription', 'tradePlanTemplate', 'lifecycle', 'setup', 'setupFilters', 'options', 'entries', 'exits']:
  print("\n\n--------------------------------------------------------\n\n")
  print("Selected column = {}".format(selected))
  keyvals = {}
  keylist=[]
  _tmp = p1.df_stkLim[selected].unique()
  for i in range(len(_tmp)):
    try:
      loaded_json = json.loads(_tmp[i])
      for key in loaded_json.keys():
        if key not in keylist:
          keylist += [key]
          keyvals[key] = loaded_json[key]
          if type(loaded_json[key]) is dict:
            for key1 in loaded_json[key].keys():
              if key1 not in keylist:
                keylist += [key+' :: '+key1]
                #keyvals[key1] = loaded_json[key][key1]
                if type(loaded_json[key][key1]) is dict:
                  for key2 in loaded_json[key][key1].keys():
                    if key2 not in keylist:
                      keylist += [key+' :: '+key1+' :: '+key2]
                      #keyvals[key2] = loaded_json[key][key1][key2]
                      if type(loaded_json[key][key1][key2]) is dict:
                        print("INCEPTION!!!!")
    except:
      pass
  
  pprint(keylist)
  print("\n\n")
  pprint(keyvals)


def get_json_dict(x):
  try:
    return json.loads(x)
  except:
    return np.nan
   
_tmp1 = p1.df_stkLim['exits'].apply(lambda x: np.nan if x is np.nan else get_json_dict(x))
_tmp = p1.df_stkLim['exits'].apply(lambda x: np.nan if x is np.nan else get_json_keysLen(x))
#pprint(_tmp1 [_tmp==7][2])

#66 for 7
#1673 for 6
#332 for 0
"""


import os
import re
import json
import numpy as np
import pandas as pd
import seaborn as sns
from string import digits
from google.colab import files
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import datetime
import os
from pprint import pprint
from IPython.core.display import display, HTML
from IPython.display import HTML
import plotly.express as px
from google.colab import data_table
data_table.enable_dataframe_formatter()
import functools
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import random
from scipy import stats
from scipy.stats import skew, mode
from IPython.display import Javascript

# Avoids scroll-in-the-scroll in the entire Notebook
def resize_colab_cell():
  #print("Enter Resize")
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
  #print("Resized")
#get_ipython().events.register('pre_run_cell', resize_colab_cell)

line_x_axis_cat = ['reduced_symbols','tradePlanTemplate','full_symbols','setup_type','setupFilters_type','entries_type','exits_types','strategy','strategydescription']
line_x_axis = ['closedate','Datewise','reduced_symbols','realizedpnl',' percent_pnl','tradePlanTemplate','costbasis','quantity','full_symbols','time-in-trade','trade_rate','setup_type','setupFilters_type','entries_type','exits_types','strategy','strategydescription','time_bracket']
line_y_axis = ['realizedpnl', ' percent_pnl', 'time-in-trade', 'trade_rate', 'Count']
line_colors = [None, 'reduced_symbols','tradePlanTemplate','full_symbols','setup_type','setupFilters_type','entries_type','exits_types','strategy','strategydescription']
axis_options = ['realizedpnl',' percent_pnl','costbasis','quantity','closedate','strategy','strategydescription','tradePlanTemplate','Day of Week','reduced_symbols','full_symbols','time-in-trade','trade_rate','W/L','Long/Short Trades','Datewise','M','hm','time_bracket','setup_type','setupFilters_type','entries_type','exits_types']
ctgr_options = [None, 'W/L','reduced_symbols','full_symbols','Long/Short Trades',' percent_pnl','trade_rate','strategy','strategydescription','tradePlanTemplate','time-in-trade','realizedpnl','Day of Week','M','hm']
symb_options = [None, 'W/L','reduced_symbols','full_symbols','Long/Short Trades','strategy','strategydescription','tradePlanTemplate']
size_options = [None, 'Count' ,'trade_rate','time-in-trade','realizedpnl',' percent_pnl']
fcet_options = [None, 'W/L', 'Long/Short Trades']
hver_options = [None, 'strategy','tradePlanTemplate','lifecycle_det','setup_type','setup_det','setupFilters_type','setupFilters_det','options_det','entries_type','entries_det','exits_types','exits_dict','strategydescription','realizedpnl','time-in-trade','trade_rate','full_symbols','reduced_symbols','Day of Week']

result_cats = ['realizedpnl', ' percent_pnl', 'time-in-trade', 'trade_rate']


plt.rcParams["figure.figsize"] = [15, 9]

clean_symbols = lambda x: re.search(r"[a-z]*", x, re.IGNORECASE).group()

cgr = lambda x: "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])

cnv = lambda x: (x-x%1)*100+(x%1)*60

get_json_dict = lambda x: json.loads(x)
get_json_keys = lambda x: [k for k in get_json_dict(x).keys()]
get_json_keysLen = lambda x: len([k for k in get_json_dict(x).keys()])

def get_json_key1(x):
  try:
    return get_json_keys(x)[0]
  except:
    return np.nan

get_json_val1 = lambda x: np.nan if get_json_key1(x) is np.nan else get_json_dict(x)[get_json_key1(x)]



def clean_options(inp):
  try:
    stk_opts = []
    for ckey in inp['optionChoices']['choices'].keys():
      choices = inp['optionChoices']['choices'][ckey]
      stk_opts += [clean_symbols(x) for x in choices]
    stk_opts = list(dict.fromkeys(stk_opts))
    inp['optionChoices'] = stk_opts
    return inp
  except:
    try:
      print(list(inp['optionChoices']['choices'].keys())[0])
    except:
      pass
    return inp

def time_bins(inp, bins=1440):

  time_slots = cnv(np.arange(24/bins, 24+24/bins, 24/bins)).astype(int).tolist()
  #lbls = cnv(np.arange(0, 24, 24/bins)).astype(int).astype(str).tolist()
  lbls = np.arange(0, 24, 24/bins).tolist()

  for i in range(len(time_slots)):
    if inp<time_slots[i]:
      return lbls[i]
  return np.nan

def time_bins_lim(inp, bins=96):

  time_slots = cnv(np.arange(24/bins, 24+24/bins, 24/bins)).astype(int).tolist()
  #lbls = cnv(np.arange(0, 24, 24/bins)).astype(int).astype(str).tolist()
  lbls = np.arange(0, 24, 24/bins).tolist()

  for i in range(len(time_slots)):
    if inp<time_slots[i]:
      return lbls[i]
  return np.nan

def dep_time_bins(inp, bins=48):
  time_slots = []
  lbls = []
  step = 24/bins
  for i in range(bins):
    pt, nt = i*step, (i+1)*step
    pt, nt = int((pt-pt%1)*100+(pt%1)*60), int((nt-nt%1)*100+(nt%1)*60)
    time_slots += [nt]
    prev_time = str(pt)[:2]+':'+str(pt)[2:]
    next_time = str(nt)[:2]+':'+str(nt)[2:]
    if bins>144:
      lbls += [prev_time]
    else:
      lbls += [prev_time+' to '+next_time]
      
  for i in range(len(time_slots)):
    if inp<time_slots[i]:
      return lbls[i]
  return np.nan

def process_stock_list(df, limit_list):
  selected_stocks = []
  if limit_list is () :
    limit_list = ['All']
  for selection in limit_list:
    if selection == 'None':
      pass
    elif selection == 'All':
      selected_stocks += df.reduced_symbols.unique().tolist()
    elif selection == 'Quick Rising':
      selected_stocks += df.groupby('reduced_symbols')['realizedpnl'].max().sort_values().reset_index()['reduced_symbols'].iloc[-4:].tolist()
    elif selection == 'Worst':
      selected_stocks += df.groupby('reduced_symbols')['realizedpnl'].min().sort_values().reset_index()['reduced_symbols'].iloc[:2].tolist()
    elif selection == 'Safest':
      selected_stocks += df.groupby('reduced_symbols')['realizedpnl'].min().sort_values().reset_index()['reduced_symbols'].iloc[-4:].tolist()
    elif selection == 'Most Traded':
      selected_stocks += df['reduced_symbols'].value_counts().reset_index()['index'].iloc[0:6].tolist()
    elif selection == 'Most Impactful':
      _tmp = df.groupby('reduced_symbols')['realizedpnl'].sum().sort_values().reset_index()
      selected_stocks += np.concatenate([ _tmp['reduced_symbols'].iloc[:2].to_numpy(), _tmp['reduced_symbols'].iloc[-2:].to_numpy() ]).tolist()
    else:
      selected_stocks += [selection]
  selected_stocks = list(dict.fromkeys(selected_stocks))
  return selected_stocks

def filter_nans(inp):
  for col in inp.columns:
    inp = inp[inp[col]!=np.nan]
    inp = inp[inp[col]!=None]
    inp = inp.dropna()
  return inp

def filter_outliers(inp, columns_to_check = ['realizedpnl']):
  for col in columns_to_check:
    inp = inp[(np.abs(stats.zscore(inp[col])) < 3)]
  return inp






def time_dist_plots(wdf):
  stkslcts = wdf.reduced_symbols.unique()
  ax = wdf[wdf['reduced_symbols']==stkslcts[0]].sort_values('time_bracket').plot.scatter(x="time_bracket", y="realizedpnl",rot=45, color = cgr(None), label=stkslcts[0], alpha=0.7)
  plt.locator_params(axis='x', nbins=6)
  for i in range(1,len(stkslcts)):
    wdf[wdf['reduced_symbols']==stkslcts[i]].sort_values('time_bracket').plot.scatter(x="time_bracket", y="realizedpnl",rot=45, color = cgr(None), label=stkslcts[i], ax=ax, alpha=0.7)
  #plt.grid(b=True, which='major')
  #ax.set_yscale('symlog', linthreshy=600)
  if np.abs(wdf['realizedpnl'].max())>400:
    plt.ylim(-300,300)
  plt.title('Trades Distribution w.r.t. Time Brackets (UTC)')

  plt.show()
  """
  ax = wdf[['time_bracket']].value_counts().sort_index().plot.bar(stacked=True, rot=45, color = cgr(None), label=stkslcts[0], alpha=1)
  plt.grid(b=True, which='major')
  plt.show()
  """
  wdf[['time_bracket_lim','reduced_symbols','Count']].groupby(['reduced_symbols','time_bracket_lim']).sum().reset_index().pivot(index='time_bracket_lim', columns='reduced_symbols', values='Count').replace({np.nan:0}).plot.bar(stacked=True, rot=45)
  plt.grid(b=True, which='major')
  plt.title("Number of trades made during each time bracket")
  #plt.locator_params(axis='x', nbins=6)
  plt.show()

def cumulative_plots(DF):
  wdf = DF.copy()
  stkslcts = wdf.reduced_symbols.unique()
  for stock in stkslcts:
    cum_stocks = lambda x: x['realizedpnl'] if x['reduced_symbols']==stock else 0
    r_stocks = lambda x: x['trade_rate'] if x['reduced_symbols']==stock else 0
    pos_vals = lambda x: x['c-'+stock] if x['c-'+stock]>0 else 0
    neg_vals = lambda x: x['c-'+stock] if x['c-'+stock]<0 else 0
    wdf[stock] = wdf[['realizedpnl', 'reduced_symbols']].apply(cum_stocks, axis=1)
    wdf['c-'+stock] = wdf[stock].cumsum()
    wdf['r-'+stock] = wdf[['trade_rate', 'reduced_symbols']].apply(r_stocks, axis=1)
    wdf['pos-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(pos_vals, axis=1)
    wdf['neg-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(neg_vals, axis=1)
  _tmp = ['pos-'+x for x in stkslcts]
  wdf['pos_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf['neg_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf.set_index('closedate', inplace=True)

  _tmp = ['pnl_accumulated']+['c-'+x for x in stkslcts]
  _rtmp = ['r-'+x for x in stkslcts]
  ax = wdf[_tmp].plot(kind="line", rot=45)
  plt.grid(b=True, which='major')
  ax.set_yscale('symlog', linthreshy=600)
  plt.title('Cumulative PNL of Trades (Total and symbol-wise)')
  plt.show()

  #wdf[['reduced_symbols','realizedpnl']].reset_index().pivot_table(index = 'closedate',columns='reduced_symbols',values='realizedpnl')
  #_tmp = wdf[['reduced_symbols','realizedpnl','Datewise']].groupby(['Datewise','reduced_symbols']).sum().reset_index().pivot(index='Datewise', columns='reduced_symbols', values='realizedpnl').replace({np.nan:0})
  _tmp = wdf[['reduced_symbols','realizedpnl']].reset_index().groupby(['closedate','reduced_symbols']).sum().reset_index().pivot(index='closedate', columns='reduced_symbols', values='realizedpnl').replace({np.nan:0})
  for col in _tmp.columns:
    _tmp[col] = _tmp[col].cumsum()
  #ax = _tmp.plot.bar(stacked=True, rot=45)
  ax = _tmp.plot.area(stacked=False, rot=45)
  wdf['realizedpnl'].cumsum().plot.line( linestyle = '-', linewidth = '2', style=['b--'], ax=ax)
  plt.locator_params(axis='x', nbins=6)
  plt.tick_params(
      axis='x',          # changes apply to the x-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,         # ticks along the top edge are off
      labelbottom=False) # labels along the bottom edge are off
  #ax.set_yscale('symlog', linthreshy=600)
  plt.grid()
  plt.title('Cumulative PNL of Trades (Total and symbol-wise)')
  plt.show()

def PNL_Spread(wdf):
  ax = wdf.plot.scatter(x="reduced_symbols", y="realizedpnl")
  plt.grid(b=True, which='major')
  ax.set_yscale('symlog', linthreshy=100)
  plt.title('Realized_pnl Spread vs Symbols')
  plt.show()

def returns_indicator(wdf):
  stat = wdf[['strategy','reduced_symbols','realizedpnl']].groupby(['strategy','reduced_symbols']).sum()
  stat['color'] = stat['realizedpnl']
  stat['size'] = np.sqrt(np.abs(stat['realizedpnl']))
  while (stat['size'].max()<700):
    stat['size'] = np.power(stat['size'],1.2)
  stat['color'][stat['realizedpnl']>0] = 'Green'
  stat['color'][stat['realizedpnl']<=0] = 'Red'
  stat.reset_index(inplace=True)
  stat.plot.scatter(x="reduced_symbols", y="strategy", c='color', s='size')
  plt.title("Strategy-wise returns indicator")
  plt.show()
  #stat

  stat = wdf[['side','reduced_symbols','realizedpnl']].groupby(['side','reduced_symbols']).sum()
  stat['color'] = stat['realizedpnl']
  stat['size'] = np.sqrt(np.abs(stat['realizedpnl']))
  while (stat['size'].max()<700):
    stat['size'] = np.power(stat['size'],1.2)
  stat['color'][stat['realizedpnl']>0] = 'Green'
  stat['color'][stat['realizedpnl']<=0] = 'Red'
  stat.reset_index(inplace=True)
  stat.plot.scatter(x="reduced_symbols", y="side", c='color', s='size')
  plt.title("Side-wise returns indicator")
  plt.show()

def make_area_plots (DF, pos_neg=True):
  wdf = DF.copy()
  stkslcts = wdf.reduced_symbols.unique()
  for stock in stkslcts:
    cum_stocks = lambda x: x['realizedpnl'] if x['reduced_symbols']==stock else 0
    r_stocks = lambda x: x['trade_rate'] if x['reduced_symbols']==stock else 0
    pos_vals = lambda x: x['c-'+stock] if x['c-'+stock]>0 else 0
    neg_vals = lambda x: x['c-'+stock] if x['c-'+stock]<0 else 0
    wdf[stock] = wdf[['realizedpnl', 'reduced_symbols']].apply(cum_stocks, axis=1)
    wdf['c-'+stock] = wdf[stock].cumsum()
    wdf['r-'+stock] = wdf[['trade_rate', 'reduced_symbols']].apply(r_stocks, axis=1)
    wdf['pos-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(pos_vals, axis=1)
    wdf['neg-'+stock] = wdf[['c-'+stock, 'reduced_symbols']].apply(neg_vals, axis=1)
  _tmp = ['pos-'+x for x in stkslcts]
  wdf['pos_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf['neg_acm'] = wdf[_tmp].sum(axis=1)
  _tmp = ['neg-'+x for x in stkslcts]
  wdf.set_index('closedate', inplace=True)

  data = wdf[_tmp][wdf[_tmp].sum(axis=1)<0]
  #plt.locator_params(axis='x', nbins=10)
  #ax = data.plot.bar(stacked=True)
  data_perc = data.divide(data.sum(axis=1), axis=0)
  ax = data_perc.plot.area(stacked=True, alpha=0.7)
  wdf[['neg_acm']].plot(kind='line', secondary_y=True, linestyle = '-', linewidth = '2', style=['r--'], ax=ax)
  plt.title('Negative Area Plot')
  plt.show()

  _tmp = ['pos-'+x for x in stkslcts]
  data = wdf[_tmp][wdf[_tmp].sum(axis=1)>0]
  data_perc = data.divide(data.sum(axis=1), axis=0)*-1
  ax = data_perc.plot.area(stacked=True)
  wdf[['pos_acm']].plot(kind='line', secondary_y=True, linestyle = '-', linewidth = '2', style=['b--'], ax=ax)
  plt.title('Positive Area Plot')
  plt.show()


global_json_var = [] # to keep a tab of columns

def get_var():
  global global_json_var
  return global_json_var
def set_var():
  global global_json_var
  global_json_var = []

def extract_roots(inp):
  try:
    inp = json.loads(inp)
  except:
    print(inp)
    return ''
  roots = ''
  for key in inp.keys():
    roots += str(key)
    roots += ', '
  return roots[:-2]

def extract_leaves_list(inp, root=None):
  outs = []
  if isinstance(inp, dict):
    for key in inp.keys():
      if isinstance(inp[key], dict):
        if root is None:
          outs += extract_leaves_list(inp[key], key)
        else:
          outs += extract_leaves_list(inp[key], root)
      else:
        if (root is None) or (root is ''):
          outs += [ str(key)+' == '+str(inp[key]) ]
        else:
          try:
            if key[4]=='-':
              outs += [ str(root)+' :: '+str(root)+' == '+str(inp[key]) ]
            else:
              outs += [ str(root)+' :: '+str(key)+' == '+str(inp[key]) ]
          except:
            outs += [ str(root)+' :: '+str(key)+' == '+str(inp[key]) ]
    return outs
  else:
    return []

def extract_leaves(inp, root=None, filters=[]):
  global global_json_var
  try:
    inp = json.loads(inp)
  except:
    print(inp)
    return {}
  inp = extract_leaves_list(inp, root)
  for filt in filters:
    _c = inp.copy()
    for item in _c:
      if filt in item:
        inp.remove(item)
  out_dict = {}
  for entry in inp:
    entry = entry.split(' == ')
    if entry[0] not in global_json_var:
      global_json_var += [entry[0]]
    try:
      out_dict[entry[0]] = float(entry[1])
    except:
      out_dict[entry[0]] = str(entry[1])
  #print(out_dict)
  return out_dict
  #return inp
  #return '\n'.join(inp)

keyval_key = None
def set_keyval(key):
  global keyval_key
  keyval_key = key
def keyval_finderr(inp):
  try:
    return inp[keyval_key]
  except:
    return np.nan


class stock_positions:
  df_inp = pd.DataFrame()
  df_full = pd.DataFrame()
  df_liveLim = pd.DataFrame()
  df_dateLim = pd.DataFrame()

  dates_range = []
  selector = None

  persistent_plot_params=[]
  persistent_chk = ''

  json_column_keys_dict = {}
  json_options = []
  global global_json_var

  debug = None
  def __init__(self, filename='', trades_filter=10):
    if os.path.exists(filename):
      self.df_inp = pd.read_csv(filename)
    else:
      uploaded = files.upload()
      filename = list(uploaded.keys())[0]
      self.df_inp = pd.read_csv(filename)
    ss_i = 0

    # Convert date columns to pandas datetime format
    self.df_inp['entrydate'], self.df_inp['closedate'] = pd.to_datetime(self.df_inp['entrydate'], utc=True), pd.to_datetime(self.df_inp['closedate'], utc=True)

    # Make an int column to use as reference index 
    self.df_inp['idxdate'] = self.df_inp['entrydate'].dt.strftime("%y%m%d%H%M%S").astype(int)
    self.df_inp.sort_values('idxdate', inplace=True)
    self.df_inp.set_index('idxdate', inplace=True)

    # Clean the symbol column
    self.df_inp['reduced_symbols'] = self.df_inp['symbol'].apply(clean_symbols)

    # Replace less-traded stocks as 'OTHER'
    self.df_inp['full_symbols'] = self.df_inp['reduced_symbols']  # Keep a copy of the column
    _tmp = self.df_inp['reduced_symbols'].value_counts().reset_index()
    other_stocks = _tmp[_tmp['reduced_symbols']<trades_filter]['index'].unique().tolist()
    for stk in other_stocks:
      self.df_inp['reduced_symbols'].replace({stk: 'OTHER'}, inplace=True)
    
    # Add time-in-trade column and handle faulty rows, replacing Faulty Values with default (1) or removing the respective rows
    self.df_inp['time-in-trade'] = (self.df_inp['closedate'] - self.df_inp['entrydate']).astype('timedelta64[s]')
    self.df_inp = self.df_inp[self.df_inp['time-in-trade'] > 0]
    #self.df_inp['time-in-trade'][ self.df_inp['time-in-trade']<=0 ] = 1

    # Make a working copy, and delete extra columns in inp to make total<=20 (for colab display)
    self.df_full = self.df_inp.copy()
    self.df_inp.drop('symbol', inplace=True, axis=1)
    self.df_inp.drop('full_symbols', inplace=True, axis=1)

    # Add a rates and win/loss columns
    rate_calc = lambda x: np.round(x['realizedpnl']/x['time-in-trade'],4)
    self.df_full['trade_rate'] = self.df_full[['realizedpnl', 'time-in-trade']].apply(rate_calc, axis=1)
    win_chk = lambda x: 'win' if x>0 else 'loss'
    self.df_full['W/L'] = self.df_full['realizedpnl'].apply(win_chk)

    # Convert to long/short sides
    LScnv = lambda x: 'short' if x=='sell_short' else 'long'
    self.df_full['Long/Short Trades'] = self.df_full['side'].apply(LScnv)

    # Add additional columns for later grouping and filtering operations
    self.df_full['E_Dates'] = self.df_full['entrydate'].dt.date
    self.df_full['Datewise'] = self.df_full['closedate'].dt.date
    self.df_full['C_Time'] = self.df_full['closedate'].dt.time
    self.df_full['H'] = self.df_full['entrydate'].dt.strftime("%H").astype(int)
    self.df_full['D'] = self.df_full['entrydate'].dt.strftime("%d").astype(int)
    self.df_full['M'] = self.df_full['entrydate'].dt.strftime("%m").astype(int)
    self.df_full['hm'] = self.df_full['entrydate'].dt.strftime("%H%M"[:4]).astype(int)
    self.df_full['hms'] = self.df_full['entrydate'].dt.strftime("%H:%M:%S")
    self.df_full['Day of Week'] = self.df_full['entrydate'].dt.day_name()
    self.df_full['time_bracket'] = self.df_full['hm'].apply(time_bins)
    self.df_full['time_bracket_lim'] = self.df_full['hm'].apply(time_bins_lim)
    self.df_full['Count'] = 1

    # Translate JSON columns to respective details
    
    set_var()
    self.df_full['lifecycle_det'] = self.df_full['lifecycle'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x))
    self.json_column_keys_dict['lifecycle'] = get_var().copy()
    self.expand_into_columns('lifecycle_det', self.json_column_keys_dict['lifecycle'])
    

    set_var()
    self.df_full['setup_type'] = self.df_full['setup'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
    self.df_full['setup_det'] = self.df_full['setup'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x, filters=['pattern']))
    self.json_column_keys_dict['setup'] = get_var().copy()
    self.expand_into_columns('setup_det', self.json_column_keys_dict['setup'])

    set_var()
    self.df_full['setupFilters_type'] = self.df_full['setupFilters'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
    self.df_full['setupFilters_det'] = self.df_full['setupFilters'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x))
    self.json_column_keys_dict['setupFilters'] = get_var().copy()
    self.expand_into_columns('setupFilters_det', self.json_column_keys_dict['setupFilters'])

    set_var()
    self.df_full['options_det'] = self.df_full['options'].apply(lambda x: np.nan if x is np.nan else clean_options(extract_leaves(x)))
    self.json_column_keys_dict['options'] = get_var().copy()
    self.expand_into_columns('options_det', self.json_column_keys_dict['options'])

    set_var()
    self.df_full['entries_type'] = self.df_full['entries'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
    self.df_full['entries_det'] = self.df_full['entries'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x))
    self.json_column_keys_dict['entries'] = get_var().copy()
    self.expand_into_columns('entries_det', self.json_column_keys_dict['entries'])

    set_var()
    self.df_full['exits_types'] = self.df_full['exits'].apply(lambda x: np.nan if x is np.nan else extract_roots(x))
    self.df_full['exits_dict'] = self.df_full['exits'].apply(lambda x: np.nan if x is np.nan else extract_leaves(x, filters=['paused', 'tacticId']))
    self.json_column_keys_dict['exits'] = get_var().copy()
    self.expand_into_columns('exits_dict', self.json_column_keys_dict['exits'])

    for key in self.json_column_keys_dict.keys():
      self.json_options += self.json_column_keys_dict[key]
    
    # Set the variable defining possible range of date selection
    self.dates_range = self.df_full.E_Dates.unique()

  def expand_into_columns(self, inp_col, col_list):
    for col in col_list:
      #keyval_finder = lambda x: (x[col] if col in x.keys() else np.nan) if isinstance(x, dict) else np.nan
      keyval_finder = lambda x: (x[col] if col in x.keys() else -1) if isinstance(x, dict) else -1
      self.df_full[col] = self.df_full[inp_col].apply(keyval_finder)
    self.df_full = self.df_full.copy()
  def make_interactive_date_selector(self):
    self.selector = widgets.SelectionRangeSlider(options=self.dates_range, index=(0, len(self.dates_range)-1), description='Period: ', layout={"width": "900px"}, disabled=False)
    display(self.selector)
    button = widgets.Button(description="Update Dates")
    output = widgets.Output()
    display(button, output)
    button.on_click(functools.partial(self.limit_dates))
    self.limit_dates(None)

  def splice_and_dice(self, bckTest ,start, end, limit_list, filt_chk, selected_plot):
    if bckTest == 'All Data':
      self.df_liveLim = self.df_full
    elif bckTest == 'Only Live':
      self.df_liveLim = self.df_full[self.df_full['accountId']!='BACKTEST']
    else:
      self.df_liveLim = self.df_full[self.df_full['accountId']=='BACKTEST']
    if (start>self.df_liveLim['E_Dates'].iloc[-1]) or (end<self.df_liveLim['E_Dates'].iloc[0]) or (start>end):
      pass
    else:
      # find start/end indices
      i_start, i_end = np.where(self.df_liveLim.E_Dates >= start)[0][0], np.where(self.df_liveLim.E_Dates <= end)[0][-1]
      if (i_start >= i_end):
        print("No data found in range, period not updated")
      else:
        # apply indices
        self.df_dateLim = self.df_liveLim.iloc[i_start:i_end].copy()
        # Recalculate cumulative pnl
        self.df_dateLim['pnl_accumulated'] = self.df_dateLim.realizedpnl.cumsum()
        #print("\rStart Date: {}\t\tEnd Date: {}".format(i_start,i_end), end="\n")
        
    limit_list = process_stock_list(self.df_dateLim, limit_list)
    self.df_stkLim = self.df_dateLim[self.df_dateLim['reduced_symbols'].isin(limit_list)].copy()
    #print('Selected stocks for analysis: {}'.format(limit_list))
    if filt_chk:
      self.df_stkLim = filter_outliers(self.df_stkLim, columns_to_check = ['realizedpnl', 'time-in-trade', 'trade_rate'])
      #print('Filtered outliers in {}'.format(['realizedpnl', 'time-in-trade', 'trade_rate']))
    #print(limit_list)
    self.df_stkLim['pnl_accumulated'] = self.df_stkLim.realizedpnl.cumsum()

    selector_options = ['Cumulative Plots', 'Time Distribution Plots', 'PNL Spread vs Symbols', 'Returns Indicator Plots', 'Profit/Loss Area Plots', "Interactive Scatter Plot", "Interactive 3D Scatter Plot", "Interactive Aggregate Plot", "Interactive Parallel Plot"]
    if selected_plot == selector_options[0]:
      cumulative_plots(self.df_stkLim)
      resize_colab_cell()
    elif selected_plot == selector_options[1]:
      time_dist_plots(self.df_stkLim)
      resize_colab_cell()
    elif selected_plot == selector_options[2]:
      PNL_Spread(self.df_stkLim)
      resize_colab_cell()
    elif selected_plot == selector_options[3]:
      returns_indicator(self.df_stkLim)
      resize_colab_cell()
    elif selected_plot == selector_options[4]:
      make_area_plots(self.df_stkLim)
      resize_colab_cell()
    elif selected_plot == selector_options[5]:
      if not self.persistent_chk==selected_plot:
        _ = interact(self.make_scatter, 
                     xaxis      =widgets.Dropdown( options=axis_options, value=axis_options[0], description='X-Axis:', disabled=False ),
                     yaxis      =widgets.Dropdown( options=axis_options, value=axis_options[9], description='Y-Axis:', disabled=False ), 
                     size_col   =widgets.Dropdown( options=size_options, value=size_options[0], description='Size Parameter:', disabled=False ),
                     color_col  =widgets.Dropdown( options=ctgr_options, value=ctgr_options[12], description='Colors Category:', disabled=False ),
                     symbol_col =widgets.Dropdown( options=symb_options, value=symb_options[0], description='Symbols Category:', disabled=False ),
                     facet_col  =widgets.Dropdown( options=fcet_options, value=fcet_options[0], description='Facet Comparison:', disabled=False ),
                     hover_col  =widgets.Dropdown( options=hver_options, value=hver_options[0], description='Hover Info:', disabled=False ))
        self.persistent_chk=selected_plot
      else:
        _ = interact(self.make_scatter, 
                     xaxis      =widgets.Dropdown( options=axis_options, value=self.persistent_plot_params[0], description='X-Axis:', disabled=False ),
                     yaxis      =widgets.Dropdown( options=axis_options, value=self.persistent_plot_params[1], description='Y-Axis:', disabled=False ), 
                     size_col   =widgets.Dropdown( options=size_options, value=self.persistent_plot_params[2], description='Size Parameter:', disabled=False ),
                     color_col  =widgets.Dropdown( options=ctgr_options, value=self.persistent_plot_params[3], description='Colors Category:', disabled=False ),
                     symbol_col =widgets.Dropdown( options=symb_options, value=self.persistent_plot_params[4], description='Symbols Category:', disabled=False ),
                     facet_col  =widgets.Dropdown( options=fcet_options, value=self.persistent_plot_params[5], description='Facet Comparison:', disabled=False ),
                     hover_col  =widgets.Dropdown( options=hver_options, value=self.persistent_plot_params[6], description='Hover Info:', disabled=False ))
    
    elif selected_plot == selector_options[6]:
      if not self.persistent_chk==selected_plot:
        _ = interact(self.make_3dscatter, 
                     xaxis      =widgets.Dropdown( options=axis_options, value=axis_options[0], description='X-Axis:', disabled=False ),
                     yaxis      =widgets.Dropdown( options=axis_options, value=axis_options[9], description='Y-Axis:', disabled=False ), 
                     zaxis      =widgets.Dropdown( options=axis_options, value=axis_options[8], description='Z-Axis:', disabled=False ),
                     size_col   =widgets.Dropdown( options=size_options, value=size_options[0], description='Size Parameter:', disabled=False ),
                     color_col  =widgets.Dropdown( options=ctgr_options, value=ctgr_options[12], description='Colors Category:', disabled=False ),
                     symbol_col =widgets.Dropdown( options=symb_options, value=symb_options[0], description='Symbols Category:', disabled=False ),
                     hover_col  =widgets.Dropdown( options=hver_options, value=hver_options[0], description='Hover Info:', disabled=False ))
        self.persistent_chk=selected_plot
      else:
        _ = interact(self.make_3dscatter, 
                     xaxis      =widgets.Dropdown( options=axis_options, value=self.persistent_plot_params[0], description='X-Axis:', disabled=False ),
                     yaxis      =widgets.Dropdown( options=axis_options, value=self.persistent_plot_params[1], description='Y-Axis:', disabled=False ), 
                     zaxis      =widgets.Dropdown( options=axis_options, value=self.persistent_plot_params[2], description='Z-Axis:', disabled=False ), 
                     size_col   =widgets.Dropdown( options=size_options, value=self.persistent_plot_params[3], description='Size Parameter:', disabled=False ),
                     color_col  =widgets.Dropdown( options=ctgr_options, value=self.persistent_plot_params[4], description='Colors Category:', disabled=False ),
                     symbol_col =widgets.Dropdown( options=symb_options, value=self.persistent_plot_params[5], description='Symbols Category:', disabled=False ),
                     hover_col  =widgets.Dropdown( options=hver_options, value=self.persistent_plot_params[6], description='Hover Info:', disabled=False ))
    
    elif selected_plot == selector_options[7]:
      if not self.persistent_chk==selected_plot:
        _ = interact(self.make_line, 
                     xaxis      =widgets.Dropdown( options=line_x_axis, value=line_x_axis[0], description='X-Axis:', disabled=False ),
                     yaxis      =widgets.Dropdown( options=line_y_axis, value=line_y_axis[0], description='Y-Axis:', disabled=False ),
                     color_col  =widgets.Dropdown( options=line_colors, value=line_colors[0], description='Colors Category:', disabled=False ))
        self.persistent_chk=selected_plot
      else:
        _ = interact(self.make_line, 
                     xaxis      =widgets.Dropdown( options=line_x_axis, value=self.persistent_plot_params[0], description='X-Axis:', disabled=False ),
                     yaxis      =widgets.Dropdown( options=line_y_axis, value=self.persistent_plot_params[1], description='Y-Axis:', disabled=False ), 
                     color_col  =widgets.Dropdown( options=line_colors, value=self.persistent_plot_params[2], description='Colors Category:', disabled=False ))
    
    elif selected_plot == selector_options[8]:
      if not self.persistent_chk==selected_plot:
        _ = interact(self.make_parallel, 
                     #color_col  =widgets.Dropdown( options=ctgr_options, value=ctgr_options[0], description='Color Category:', disabled=False ),
                     rdims      =widgets.SelectMultiple(options=result_cats, description='Output Parameters:', disabled=False, value = (result_cats[0],) ),
                     dims       =widgets.SelectMultiple(options=self.json_options, description='Configuration Parameters:', disabled=False, value = (self.json_options[0],) ))
        self.persistent_chk=selected_plot
      else:
        try:
          _ = interact(self.make_parallel, 
                     #color_col  =widgets.Dropdown( options=ctgr_options, value=self.persistent_plot_params[0], description='Color Category:', disabled=False ),
                     rdims      =widgets.SelectMultiple(options=result_cats, value =self.persistent_plot_params[1], description='Output Parameters:', disabled=False),
                     dims       =widgets.SelectMultiple(options=self.json_options, value =self.persistent_plot_params[1], description='Configuration Parameters:', disabled=False))
        except:
          _ = interact(self.make_parallel, 
                     #color_col  =widgets.Dropdown( options=ctgr_options, value=ctgr_options[0], description='Color Category:', disabled=False ),
                     rdims      =widgets.SelectMultiple(options=result_cats, description='Output Parameters:', disabled=False, value = (result_cats[0],) ),
                     dims       =widgets.SelectMultiple(options=self.json_options, description='Configuration Parameters:', disabled=False, value = (self.json_options[0],) ))
    
    else:
      print("Plot selection not implemented.")
    #resize_colab_cell()
    

  def interact_sequence(self):
    selector_options_s = ['All', 'Most Traded', 'Most Impactful', 'Safest', 'Worst', 'Quick Rising'] + self.df_full.reduced_symbols.unique().tolist()
    selector_options = ['Cumulative Plots', 'Time Distribution Plots', 'PNL Spread vs Symbols', 'Returns Indicator Plots', 'Profit/Loss Area Plots', "Interactive Scatter Plot", "Interactive 3D Scatter Plot", "Interactive Aggregate Plot", "Interactive Parallel Plot"]
    resize_colab_cell()
    _ = interact( self.splice_and_dice,
                  bckTest         = widgets.RadioButtons(options=['All Data', 'Only Live', 'Only Backtest'], value='All Data', layout={'width': 'max-content'}, description='Select Data Category:', disabled=False),
                  start           = widgets.DatePicker(value=self.df_full['E_Dates'].iloc[0] ),
                  end             = widgets.DatePicker(value=self.df_full['E_Dates'].iloc[-1]),
                  limit_list      = widgets.SelectMultiple(options=selector_options_s, description='Stocks:', disabled=False, value = (selector_options_s[0],) ),
                  filt_chk        = widgets.Checkbox( value=False, description='Filter outlier trade entries?', disabled=False, indent=True ),
                  selected_plot   = widgets.ToggleButtons( options=selector_options, description='Plot Selection:', disabled=False, button_style='', tooltips=selector_options, value=selector_options[5] )    )
    
  def make_parallel(self, rdims, dims):
    self.persistent_plot_params = [rdims, dims]
    try:
      #cols = [color_col]+list(dims)
      cols = list(rdims)+list(dims)
      filt = self.df_stkLim[list(filter(None, cols))].copy()
      #filt = filter_nans(self.df_stkLim[list(filter(None, cols))].copy())
      self.debug = filt
      #fig = px.parallel_coordinates(filt, color=color_col, dimensions=list(dims), width=1600, height=800)
      fig = px.parallel_coordinates(filt, dimensions=list(dims)+list(rdims), width=1600, height=800)
      fig.update_layout(paper_bgcolor="LightSteelBlue")
      display(HTML(fig.to_html()))
    except Exception as E:
      print("Couldn't make graph due to following error: "), print(E)

  def make_scatter(self, xaxis,yaxis,size_col,color_col, symbol_col, facet_col, hover_col):
    try:
      cols = [xaxis, yaxis, size_col, color_col, symbol_col, facet_col, hover_col]
      col2 = [xaxis, yaxis, color_col, symbol_col, facet_col, hover_col]
      self.persistent_plot_params = cols
      #try:
      if (size_col in col2) and (size_col is not None):
        filt = filter_nans(self.df_stkLim[list(filter(None, col2))].copy())
        filt[size_col+'_'] = filt[size_col] 
        size_col = size_col+'_'
      else:
        filt = filter_nans(self.df_stkLim[list(filter(None, cols))].copy())
      #filt = filt.groupby([xaxis,yaxis]).agg({size_col:'sum', color_col:'min', symbol_col:'min', facet_col:'min', hover_col:'min'})
      agg_cols = [size_col, color_col, symbol_col, facet_col, hover_col]
      agg_mets = ['sum', 'first', 'first', 'first', 'first']
      agg_dict={}
      for i in range(len(agg_mets)):
        if agg_cols[i] is not None:
          agg_dict[agg_cols[i]] = agg_mets[i]
      if agg_dict != {}:
        filt = filt.groupby([xaxis,yaxis]).agg(agg_dict)
      try:
        filt[size_col] = np.abs(filt[size_col])
        filt.rename(columns={size_col: 'ABS('+size_col+')'}, inplace=True)
        size_col = 'ABS('+size_col+')'
      except:
        size_col = None
      filt = filter_nans(filt)
      self.debug = filt
      fig = px.scatter(filt.reset_index(), x=xaxis, y=yaxis, color=color_col, size=size_col, symbol=symbol_col, facet_col=facet_col, hover_data=[hover_col], width=1600, height=800)
      fig.update_layout(paper_bgcolor="LightSteelBlue")
      #display(HTML(fig.to_html()))
      #resize_colab_cell()
      display(HTML(fig.to_html()))
    except Exception as E:
      print("Couldn't make graph due to following error: "), print(E)


  def make_3dscatter(self, xaxis, yaxis, zaxis, size_col, color_col, symbol_col, hover_col):
    try:
      cols = [xaxis, yaxis, zaxis, size_col, color_col, symbol_col, hover_col]
      col2 = [xaxis, yaxis, zaxis, color_col, symbol_col, hover_col]
      self.persistent_plot_params = cols
      #try:
      if (size_col in col2) and (size_col is not None):
        filt = filter_nans(self.df_stkLim[list(filter(None, col2))].copy())
        filt[size_col+'_'] = filt[size_col] 
        size_col = size_col+'_'
      else:
        filt = filter_nans(self.df_stkLim[list(filter(None, cols))].copy())
      #filt = filt.groupby([xaxis,yaxis]).agg({size_col:'sum', color_col:'min', symbol_col:'min', hover_col:'min'})
      agg_cols = [size_col, color_col, symbol_col, hover_col]
      agg_mets = ['sum', 'first', 'first', 'first']
      agg_dict={}
      for i in range(len(agg_mets)):
        if agg_cols[i] is not None:
          agg_dict[agg_cols[i]] = agg_mets[i]
      if agg_dict != {}:
        filt = filt.groupby([xaxis,yaxis,zaxis]).agg(agg_dict)
      try:
        filt[size_col] = np.abs(filt[size_col])
        filt.rename(columns={size_col: 'ABS('+size_col+')'}, inplace=True)
        size_col = 'ABS('+size_col+')'
      except:
        size_col = None
      filt = filter_nans(filt)
      self.debug = filt
      fig = px.scatter_3d(filt.reset_index(), x=xaxis, y=yaxis, z=zaxis, color=color_col, size=size_col, symbol=symbol_col, hover_data=[hover_col], width=1600, height=800)
      fig.update_layout(paper_bgcolor="LightSteelBlue")
      #display(HTML(fig.to_html()))
      #resize_colab_cell()
      display(HTML(fig.to_html()))
    except Exception as E:
      print("Couldn't make graph due to following error: "), print(E)
  
  def make_line(self, xaxis, yaxis, color_col):
    try:
      cols = [xaxis, yaxis, color_col]
      self.persistent_plot_params = cols
      filt = filter_nans(self.df_stkLim[list(filter(None, cols))].copy())
      filt = filter_nans(filt)
      if color_col:
        filt = filt.groupby([xaxis,color_col]).agg('sum')
      else:
        filt = filt.groupby([xaxis]).agg('sum')
      filt = filter_nans(filt)
      filt.reset_index(inplace=True)
      filt = filt.sort_values(by=xaxis)
      if color_col:
        filt = filt.sort_values(by=color_col)
      filt = filt.sort_values(xaxis).reset_index(drop=True)
      self.debug = filt
      if xaxis in line_x_axis_cat:
        fig = px.scatter(filt, x=xaxis, y=yaxis, color=color_col, width=1600, height=800)
      else:
        fig = px.line(filt, x=xaxis, y=yaxis, color=color_col, width=1600, height=800)
      fig.update_layout(paper_bgcolor="LightSteelBlue")
      #resize_colab_cell()
      display(HTML(fig.to_html()))
      
    except Exception as E:
      print("Couldn't make graph due to following error: "), print(E)
    
  
  #stk_cls.df_dateLim.head()



p1 = stock_positions('closed_positions_rt.csv', 20)
p1.df_full[['full_symbols','reduced_symbols', 'realizedpnl', 'time-in-trade','trade_rate','W/L']].iloc[700:900]

p1.df_inp.iloc[700:900]
p1.interact_sequence()